In [1]:
import pygad
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score
from mlconfound.stats import partial_confound_test
from mlconfound.plot import plot_null_dist, plot_graph
from sklearn.model_selection import train_test_split

In [2]:
DATA_ALL = sio.loadmat("data/subjects_40_v6.mat")

dict_keys(['__header__', '__version__', '__globals__', 'DATA', 'FEAT', 'FEAT_N', 'LABEL', 'LABEL_VOWEL', 'SUBJECT_ID', 'SUBJECT_SKINFOLD', 'SUBJECT_VFI', 'VOWEL_REP'])

In [3]:
FEAT_N           = DATA_ALL['FEAT_N']            # Normalized features
LABEL            = DATA_ALL['LABEL']             # Labels
VFI_1            = DATA_ALL['SUBJECT_VFI']
SUBJECT_ID       = DATA_ALL['SUBJECT_ID']        # Sujbect ID
SUBJECT_SKINFOLD = DATA_ALL['SUBJECT_SKINFOLD']

In [4]:
# Gram-Schmidt
def gram_schmidt(A,norm=True,row_vect=False):
    """Orthonormalizes vectors by gram-schmidt process
    
    Parameters
    -----------
    A : ndarray,
    Matrix having vectors in its columns
    
    norm : bool,
    Do you need Normalized vectors?
    
    row_vect: bool,
    Does Matrix A has vectors in its rows?
    
    Returns 
    -------
    G : ndarray,
    Matrix of orthogonal vectors 
    
    """
    if row_vect :
        # if true, transpose it to make column vector matrix
        A = A.T
    
    no_of_vectors = A.shape[1]
    G = A[:,0:1].copy() # copy the first vector in matrix
    # 0:1 is done to to be consistent with dimensions - [[1,2,3]]
    
    # iterate from 2nd vector to number of vectors
    for i in range(1,no_of_vectors):
        
        # calculates weights(coefficents) for every vector in G
        numerator = A[:,i].dot(G)
        denominator = np.diag(np.dot(G.T,G)) #to get elements in diagonal
        weights = np.squeeze(numerator/denominator)
        
        # projected vector onto subspace G 
        projected_vector = np.sum(weights * G,
                                  axis=1,
                                  keepdims=True)
        
        # orthogonal vector to subspace G
        orthogonalized_vector = A[:,i:i+1] - projected_vector
        
        # now add the orthogonal vector to our set 
        G = np.hstack((G,orthogonalized_vector))
        
    if norm :
        # to get orthoNORMAL vectors (unit orthogonal vectors)
        # replace zero to 1 to deal with division by 0 if matrix has 0 vector
        G = G/replace_zero(np.linalg.norm(G,axis=0))
    
    if row_vect:
        return G.T
    
    return G

def replace_zero(array): 
    
    for i in range(len(array)) :
        if array[i] == 0 : 
            array[i] = 1
    return array

In [5]:
# Testing the matrix for Gram-Schmidt Orthogonization
features_all = np.zeros((0,48))
label_all    = np.zeros((0,1))
skinfold     = np.zeros((0,2))
subject_id   = np.zeros((0,1))

# subject_

for i_sub in range(40):
    features_all = np.append(features_all, FEAT_N[i_sub,0],           axis=0)
    label_all    = np.append(label_all,    LABEL[i_sub,0],            axis=0)
    skinfold     = np.append(skinfold,     SUBJECT_SKINFOLD[i_sub,0], axis=0)
    subject_id   = np.append(subject_id,   SUBJECT_ID[i_sub,0],       axis=0)
    
skinfold_avg = np.mean(skinfold, axis=1) 

In [6]:
# Move the skinfold thickness as the first column
G = np.concatenate((skinfold_avg.reshape((-1,1)), features_all), axis=1)
# print(np.shape(G))
# Perform the Gram Schmidt Orthogonization
G_test = gram_schmidt(G)
feature_G = G_test[:,1:]
print(np.shape(feature_G))

(6472, 48)


In [7]:
leftout = 1
testing_acc  = np.zeros(40)
valid_acc    = np.zeros(40)
training_acc = np.zeros(40)
p_value      = np.zeros(40)

sub_index = [1]
for sub_test in range(40):
    print('\n===Exp No. %d===\n'%(sub_test+1))
    
    sub_id = SUBJECT_ID[sub_test][0][0]
    sub_txt = "R%03d"%(int(sub_id))
    print('Test Subject %s:'%(sub_txt))
    print('VFI-1:', (VFI_1[sub_test][0][0]))
    if int(VFI_1[sub_test][0][0]) > 10:
        sub_group = 'Fatigued'
    else:
        sub_group = 'Healthy'

    # ===== Load Testing Signals =====
#     num_signal = np.shape(FEAT_N[sub_test,0])[0]    
#     X_Temp = FEAT_N[sub_test,0]
#     Y_Temp = LABEL[sub_test,0].flatten()

#     num_leftout = round(leftout*num_signal)
#     index_leftout = np.random.choice(range(num_signal), size=num_leftout, replace=False)
#     print("Left-out Test samples: ", index_leftout.size)

#     X_Test = X_Temp[index_leftout,:]
#     Y_Test = Y_Temp[index_leftout]

#     index_include = np.arange(num_signal)
#     index_include = np.delete(index_include, index_leftout)
#     print("Included Training samples: ", index_include.size)
#     X_include = X_Temp[index_include,:]
#     Y_include = Y_Temp[index_include]
    
    X_Test = feature_G[np.ravel(subject_id == sub_id),:]
    Y_Test = label_all[np.ravel(subject_id == sub_id),:]

    # ===== Load Traing Signals =====
    X_TV = np.zeros((0,48))
    Y_TV = np.zeros(0)    
    C_TV = np.zeros(0)
    for sub_train in range(40):
        if sub_train != sub_test:
            sub_id = SUBJECT_ID[sub_train][0][0]
#             x_s = FEAT_N[sub_train,0]
#             y_s = LABEL[sub_train,0].flatten()
#             c_s = np.mean(np.mean(SUBJECT_SKINFOLD[sub_train,:]), axis=1)
            x_s = feature_G[np.ravel(subject_id == sub_id),:]
            y_s = label_all[np.ravel(subject_id == sub_id),:].flatten()
            c_s = skinfold_avg[np.ravel(subject_id == sub_id)]
        
            # ===== CAN BE CONVERTED INTO A FUNCTION =====
            X_TV = np.concatenate((X_TV, x_s), axis=0)
            Y_TV = np.concatenate((Y_TV, y_s), axis=0)
            C_TV = np.concatenate((C_TV, c_s), axis=0)       

    print('# of Healthy Samples: %d'%(np.sum(Y_TV == -1)))
    print('# of Fatigued Samples: %d'%(np.sum(Y_TV == 1)))    

    # ===== Data loading and preprocessing =====
    # Training and Validation
    # Training and Validation
    X_Train, X_Valid, YC_Train, YC_Valid = train_test_split(X_TV, 
                                                            np.transpose([Y_TV, C_TV]), 
                                                            test_size=0.1, 
                                                            random_state=42)
    Y_Train, C_Train = YC_Train[:,0], YC_Train[:,1]
    Y_Valid, C_Valid = YC_Valid[:,0], YC_Valid[:,1]    
    
    clf = LinearSVC(class_weight='balanced', max_iter=1000, tol=0.0001) 
    clf.fit(X_Train, Y_Train)
    
    label_predict = clf.predict(X_Train)    
    ret = partial_confound_test(Y_Train, label_predict, C_Train, progress=True)  
    print('P value: ', ret.p)
    p_value[sub_test] = ret.p
    
    print('Training Acc: ', accuracy_score(label_predict, Y_Train))
    training_acc[sub_test] = accuracy_score(label_predict, Y_Train)

    label_predict = clf.predict(X_Valid)
    print('Validation Acc: ', accuracy_score(label_predict, Y_Valid))
    valid_acc[sub_test] = accuracy_score(label_predict, Y_Valid)

    label_predict = clf.predict(X_Test)
    print('Testing Acc: ', accuracy_score(label_predict, Y_Test))
    testing_acc[sub_test] = accuracy_score(label_predict, Y_Test)


===Exp No. 1===

Test Subject R044:
VFI-1: [0]
# of Healthy Samples: 3037
# of Fatigued Samples: 3270


Permuting: 100%|██████████| 1000/1000 [00:17<00:00, 58.43it/s]


P value:  0.994
Training Acc:  0.8757928118393234
Validation Acc:  0.8827258320126783
Testing Acc:  0.24242424242424243

===Exp No. 2===

Test Subject R041:
VFI-1: [2]
# of Healthy Samples: 3038
# of Fatigued Samples: 3270


Permuting: 100%|██████████| 1000/1000 [00:14<00:00, 68.53it/s]


P value:  0.931
Training Acc:  0.8728201514884623
Validation Acc:  0.873217115689382
Testing Acc:  0.3719512195121951

===Exp No. 3===

Test Subject R081:
VFI-1: [0]
# of Healthy Samples: 3037
# of Fatigued Samples: 3270


...

P value:  0.832
Training Acc:  0.8674062334918119
Validation Acc:  0.8462757527733756
Testing Acc:  0.42592592592592593

===Exp No. 39===

Test Subject R046:
VFI-1: [18]
# of Healthy Samples: 3202
# of Fatigued Samples: 3106


Permuting: 100%|██████████| 1000/1000 [00:07<00:00, 128.22it/s]


P value:  0.998
Training Acc:  0.8740531971111503
Validation Acc:  0.8399366085578447
Testing Acc:  0.524390243902439

===Exp No. 40===

Test Subject R049:
VFI-1: [28]
# of Healthy Samples: 3202
# of Fatigued Samples: 3106


Permuting: 100%|██████████| 1000/1000 [00:07<00:00, 125.13it/s]

P value:  0.993
Training Acc:  0.8761669896071869
Validation Acc:  0.8415213946117274
Testing Acc:  0.4024390243902439


In [8]:
np.mean(training_acc)

0.8694922945476826

In [9]:
np.mean(valid_acc)

0.8479690825795142

In [10]:
np.mean(testing_acc)

0.6339012581825741

In [11]:
np.mean(p_value)

0.9252750000000001

In [12]:
import pandas as pd

data_array = np.array([training_acc, valid_acc, testing_acc, p_value]).T
df = pd.DataFrame(data_array, columns = ['Train', 'Valid', 'Test', 'P-Value'])
print(df)

       Train     Valid      Test  P-Value
0   0.875793  0.882726  0.242424    0.994
1   0.872820  0.873217  0.371951    0.931
2   0.866279  0.870048  0.781818    0.997
3   0.876387  0.884311  0.240741    0.213
4   0.861948  0.857369  0.993827    0.999
5   0.867865  0.865293  0.612121    0.723
6   0.878478  0.871632  0.239264    0.994
7   0.864300  0.870458  1.000000    0.998
8   0.873525  0.852615  0.542683    0.987
9   0.874075  0.852615  0.411043    0.974
10  0.867007  0.844691  0.993902    0.995
11  0.870292  0.859177  0.291139    0.999
12  0.864693  0.849445  0.806061    0.971
13  0.867254  0.832278  1.000000    0.997
14  0.866103  0.836767  0.981818    0.934
15  0.865622  0.830428  0.987730    0.967
16  0.864341  0.852615  0.769697    0.902
17  0.866103  0.839937  0.763636    0.924
18  0.871212  0.833597  0.284848    0.902
19  0.863779  0.857143  0.990826    0.997
20  0.864627  0.835443  0.453947    1.000
21  0.868041  0.836767  0.581818    0.976
22  0.889183  0.866878  0.078788  

In [14]:
df.mean(axis=0)

Train      0.869492
Valid      0.847969
Test       0.633901
P-Value    0.925275
dtype: float64

In [13]:
df.to_csv('Linear_SVM_Gram-Schmidt_LOO.csv')